# Analysis of migration data: inflow and outflow, local and US-level

In [1]:
import pysal as ps
import numpy as np
import pandas as pd
counties = '/home/anna/data/FUTURES/counties/us_county.shp'
migration_table = '/home/anna/Projects/FUTURES/migration/migration_table_v4.csv'
msa_table = '/home/anna/Projects/FUTURES/data/recode_counties_MSA.txt'

## Process migration table

In [2]:
msa = pd.read_csv(msa_table, delimiter=':', usecols=[0, 2], names=['FIPS', 'msa'], dtype={"FIPS": "int64", "msa": "int64"})
msa

,FIPS,msa
0,48253,10180
1,48059,10180
2,48441,10180
3,39133,10420
4,39153,10420
...,...,...
1170,42085,49660
1171,39099,49660
1172,6115,49700
1173,6101,49700


In [7]:
df_migration = pd.read_csv(migration_table, index_col=0,  dtype={"origin": "Int64", "destination": "int64"})
df_migration = df_migration.drop(columns=['key', 'NA'])
df_migration

,origin,destination,X1990,X1991,X1992,X1993,X1994,X1995,X1996,X1997,...,X2006,X2007,X2008,X2009,X2010,X2011,X2012,X2013,X2014,X2015
1,2016,53033,59,67,1431,58,940,52,64,71,...,775,5205,21,25,23,29,28,30,40,26
2,2016,2020,45,46,1080,61,1149,40,29,37,...,1070,5264,41,55,31,31,34,29,38,35
3,2016,6073,31,71,1043,83,1149,36,22,21,...,0,4565,0,10,0,0,0,0,0,0
4,2016,53035,19,45,933,33,809,15,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2016,53053,17,12,732,24,576,0,0,0,...,0,0,0,0,0,0,10,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169419,48453,37057,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22
169420,48453,55009,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22
169421,48453,9007,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
169422,48453,54039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20


In [8]:
# sum over years
df_migration['sum'] = df_migration.loc[:,'X1990':'X2015'].sum(1)
df_migration = df_migration.drop(columns=df_migration.loc[:,'X1990':'X2015'])
df_migration = df_migration[~df_migration.origin.isin([57001, 57003, 57005, 57007])]
df_migration.loc[df_migration.origin == df_migration.destination, 'sum'] = 0
df_migration

,origin,destination,sum
1,2016,53033,10811
2,2016,2020,11668
3,2016,6073,7031
4,2016,53035,1854
5,2016,53053,1371
...,...,...,...
169419,48453,37057,22
169420,48453,55009,22
169421,48453,9007,20
169422,48453,54039,20


## Aggregate to MSA

In [9]:
# aggregate origin column
df1 = df_migration.merge(msa, left_on='origin', right_on='FIPS', how="left")
df1['msa'] = df1['msa'].fillna(df1['origin'])
df1.drop(["origin", "FIPS"], inplace=True, axis=1)
df1.rename(columns={'msa':'origin'}, inplace=True)
df1 = df1[['origin', 'destination', 'sum']]
display(df1)

,origin,destination,sum
0,2016.0,53033,10811
1,2016.0,2020,11668
2,2016.0,6073,7031
3,2016.0,53035,1854
4,2016.0,53053,1371
...,...,...,...
167257,12420.0,37057,22
167258,12420.0,55009,22
167259,12420.0,9007,20
167260,12420.0,54039,20


In [10]:
# aggregate destination column
df2 = df1.merge(msa,left_on='destination', right_on='FIPS', how="left")
df2['msa'] = df2['msa'].fillna(df2['destination'])
df2.drop(["destination", "FIPS"], inplace=True, axis=1)
df2.rename(columns={'msa':'destination'}, inplace=True)
df2 = df2[['origin', 'destination', 'sum']]
display(df2)

,origin,destination,sum
0,2016.0,42660.0,10811
1,2016.0,2020.0,11668
2,2016.0,41740.0,7031
3,2016.0,14740.0,1854
4,2016.0,42660.0,1371
...,...,...,...
167257,12420.0,49180.0,22
167258,12420.0,24580.0,22
167259,12420.0,25540.0,20
167260,12420.0,16620.0,20


Change the same MSA into 0, ignoring migration within MSA.

In [11]:
df2.loc[df2.origin == df2.destination, 'sum'] = 0
df2

,origin,destination,sum
0,2016.0,42660.0,10811
1,2016.0,2020.0,11668
2,2016.0,41740.0,7031
3,2016.0,14740.0,1854
4,2016.0,42660.0,1371
...,...,...,...
167257,12420.0,49180.0,22
167258,12420.0,24580.0,22
167259,12420.0,25540.0,20
167260,12420.0,16620.0,20


In [12]:
df3 = df2.groupby(by=["origin", "destination"])['sum'].sum().reset_index().astype('int32')
df3

,origin,destination,sum
0,1005,1005,0
1,1005,1011,1226
2,1005,1031,262
3,1005,1045,3234
4,1005,1087,102
...,...,...,...
80804,56045,56001,714
80805,56045,56005,5340
80806,56045,56011,1997
80807,56045,56033,11


## Transform to matrix

In [13]:
df3.set_index(['origin', 'destination'], inplace=True)
df_migration_MSA = df3.unstack(level=-1, fill_value=0)['sum']
df_migration_MSA

destination,1005,1011,1013,1017,1019,1023,1025,1027,1029,1031,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
origin,,,,,,,,,,,,,,,,,,,,,
1005,0,1226,0,0,0,0,0,0,0,262,...,0,0,0,0,0,0,0,0,0,0
1011,1363,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1017,0,0,0,0,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,0
1019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,0,0,0,0,0,0,0,0,0,0,...,0,30,0,2727,6308,0,37,7971,11,0
56039,0,0,0,0,0,0,0,0,0,0,...,0,1242,0,10,2855,25,0,0,0,0
56041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12,1280,8174,0,0,0,0


## Analyse infow and outflow

In [14]:
df_in_out = pd.DataFrame(index=df_migration_MSA.index)
df_in_out['inflow'] = df_migration_MSA.sum(axis=0)
df_in_out['outflow'] = df_migration_MSA.sum(axis=1)
df_in_out['max'] = df_in_out[['inflow', 'outflow']].max(axis=1)
df_in_out['percentage'] = 100 * (df_in_out['inflow'] - df_in_out['outflow']) / df_in_out['max']
df_in_out['percentage_scaled'] = (df_in_out['percentage'] / 2) + 50
df_in_out

,inflow,outflow,max,percentage,percentage_scaled
origin,,,,,
1005,17339,22539,22539,-23.071121,38.464439
1011,6299,8449,8449,-25.446798,37.276601
1013,10650,12948,12948,-17.747915,41.126043
1017,24366,26580,26580,-8.329571,45.835214
1019,19479,17729,19479,8.984034,54.492017
...,...,...,...,...,...
56037,109270,118890,118890,-8.091513,45.954243
56039,48996,77147,77147,-36.490077,31.754961
56041,43846,61698,61698,-28.934487,35.532756


## Transform to within-county migration

Compute migrants within county assuming ratio = 0 => 50% migrants stay.

In [15]:
within_migrants = (df_migration_MSA.sum(axis=1) * df_in_out['percentage_scaled']) / (100 - df_in_out['percentage_scaled'])

Update diagonal of migration matrix with these values.

In [16]:
array = df_migration_MSA.values
np.fill_diagonal(array, within_migrants)
df_migration_MSA = pd.DataFrame(array, index=df_migration_MSA.index, columns=df_migration_MSA.columns)

In [17]:
df_migration_MSA

destination,1005,1011,1013,1017,1019,1023,1025,1027,1029,1031,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
origin,,,,,,,,,,,,,,,,,,,,,
1005,14088,1226,0,0,0,0,0,0,0,262,...,0,0,0,0,0,0,0,0,0,0
1011,1363,5021,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013,0,0,9044,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1017,0,0,0,22492,0,0,0,12,0,0,...,0,0,0,0,0,0,0,0,0,0
1019,0,0,0,0,21229,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,0,0,0,0,0,0,0,0,0,0,...,0,30,0,2727,6308,101090,37,7971,11,0
56039,0,0,0,0,0,0,0,0,0,0,...,0,1242,0,10,2855,25,35897,0,0,0
56041,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12,1280,8174,0,34006,0,0


Compute probabilities:

In [18]:
df_migration_adjusted = df_migration_MSA.div(df_migration_MSA.sum(axis=1), axis=0) * 100
df_migration_adjusted = df_migration_adjusted.fillna(0)
df_migration_adjusted

destination,1005,1011,1013,1017,1019,1023,1025,1027,1029,1031,...,56027,56029,56031,56033,56035,56037,56039,56041,56043,56045
origin,,,,,,,,,,,,,,,,,,,,,
1005,38.463429,3.347257,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.715319,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1011,10.118782,37.275427,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1013,0.000000,0.000000,41.124045,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1017,0.000000,0.000000,0.000000,45.834692,0.000000,0.0,0.0,0.024454,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1019,0.000000,0.000000,0.000000,0.000000,54.492017,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.013638,0.0,1.239658,2.867533,45.954178,0.016820,3.623511,0.00500,0.000000
56039,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,1.098687,0.0,0.008846,2.525565,0.022115,31.754892,0.000000,0.00000,0.000000
56041,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.012539,1.337457,8.540918,0.000000,35.532475,0.00000,0.000000


In [19]:
df_migration_adjusted.to_csv('/home/anna/Projects/FUTURES/migration/migration_matrix_within.csv', header=True, index=True, index_label=None)
df_in_out.to_csv('/home/anna/Projects/FUTURES/migration/migration_inoutratio.csv', columns=['percentage'], header=True, index=True, index_label=None)